In [3]:
import streamlit as st

import pandas as pd
import statsmodels.api as sm 
import numpy as np


from time import  time
import plotly.express as px
from pathlib import Path
import plotly.graph_objects as go


In [4]:
data=pd.read_csv("final.csv")

In [5]:
data['sell_price'] = data['sell_price'].astype('float')
data['sold'] = data['sold'].astype('int')



In [6]:
def group_df(df):
    temp=df.groupby(['id','YearMonth','cat_id','store_id'])

    return temp


In [7]:
test = group_df(data).aggregate({'sold':np.sum,'sell_price':np.mean})

first_values = sorted(pd.DataFrame([t[0] for t in test.index])[0].unique())
def ped1(group):
    X = sm.add_constant(group['sell_price'])
    model = sm.OLS(group['sold'], X).fit()
    price_coef = model.params['sell_price']
    mean_sellprice = np.mean(group['sell_price'])
    mean_quantity = np.mean(group['sold'])
    ped = price_coef * (mean_sellprice / mean_quantity)

    return ped



C:\Users\lywt9\AppData\Local\Temp\ipykernel_16736\3596184817.py:1: FutureWarning: The provided callable <function sum at 0x000001DDEA735580> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  test = group_df(data).aggregate({'sold':np.sum,'sell_price':np.mean})
C:\Users\lywt9\AppData\Local\Temp\ipykernel_16736\3596184817.py:1: FutureWarning: The provided callable <function mean at 0x000001DDEA736660> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  test = group_df(data).aggregate({'sold':np.sum,'sell_price':np.mean})
C:\Users\lywt9\AppData\Local\Temp\ipykernel_16736\3596184817.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with Dat

In [13]:
ped=data.groupby(['cat_id']).apply(ped1)

In [14]:
food_ped=ped[0]
hobby_ped=ped[1]
house_ped=ped[2]

C:\Users\lywt9\AppData\Local\Temp\ipykernel_16736\1480201535.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  food_ped=ped[0]
C:\Users\lywt9\AppData\Local\Temp\ipykernel_16736\1480201535.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hobby_ped=ped[1]
C:\Users\lywt9\AppData\Local\Temp\ipykernel_16736\1480201535.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  house_ped=ped[2]


In [17]:
house_ped

-0.7699144192239312

In [48]:
test.index

MultiIndex([(    'FOODS_1_001_CA_1', '2011-01',     'FOODS', 'CA_1'),
            (    'FOODS_1_001_CA_1', '2011-02',     'FOODS', 'CA_1'),
            (    'FOODS_1_001_CA_1', '2011-03',     'FOODS', 'CA_1'),
            (    'FOODS_1_001_CA_1', '2011-04',     'FOODS', 'CA_1'),
            (    'FOODS_1_001_CA_1', '2011-05',     'FOODS', 'CA_1'),
            (    'FOODS_1_001_CA_1', '2011-06',     'FOODS', 'CA_1'),
            (    'FOODS_1_001_CA_1', '2011-07',     'FOODS', 'CA_1'),
            (    'FOODS_1_001_CA_1', '2011-08',     'FOODS', 'CA_1'),
            (    'FOODS_1_001_CA_1', '2011-09',     'FOODS', 'CA_1'),
            (    'FOODS_1_001_CA_1', '2011-10',     'FOODS', 'CA_1'),
            ...
            ('HOUSEHOLD_2_516_WI_3', '2015-07', 'HOUSEHOLD', 'WI_3'),
            ('HOUSEHOLD_2_516_WI_3', '2015-08', 'HOUSEHOLD', 'WI_3'),
            ('HOUSEHOLD_2_516_WI_3', '2015-09', 'HOUSEHOLD', 'WI_3'),
            ('HOUSEHOLD_2_516_WI_3', '2015-10', 'HOUSEHOLD', 'WI_3'),
    

In [68]:
def calc_ped(grouped,values,disc,food_ped,hobby_ped,house_ped):
    final=[]
    for i in range(len(values)):
        X = sm.add_constant(grouped.loc[values[i]]['sell_price'])
        model = sm.OLS(grouped.loc[values[i]]['sold'], X).fit()
        mean_sellprice = np.mean(grouped.loc[values[i]]['sell_price'])
        mean_quantity = np.mean(grouped.loc[values[i]]['sold'])
        ped = model.params['sell_price'] * (mean_sellprice / mean_quantity)
        if ped>0:
            if grouped.index[i][2]=='FOODS':
                ped=food_ped
            elif grouped.index[i][2]=='HOBBIES':
                ped=hobby_ped
            else:
                ped=house_ped
        last_quantity=grouped.loc[values[i]]['sold'].iloc[-1]
        last_price=grouped.loc[values[i]]['sell_price'].iloc[-1]
        new_price=last_price*(1-(disc/100))
        new_quantity=last_quantity*(1+ped*(last_price*((100-disc)/100)-last_price)/last_price)
        cat=grouped.loc[values[i]].index[0][1]
        state=grouped.loc[values[i]].index[0][2]
        
        final.append([values[i],cat,state,last_price,new_price,ped,last_quantity,new_quantity])
    final=pd.DataFrame(final,columns=["Product ID","Category","State ID","Original Price","Discounted Price","Predicted PED","Original Quantity","Predicted Quantity"])
    return final

discount=0
final=calc_ped(test,first_values,discount,food_ped,hobby_ped,house_ped)


In [74]:
final.shape

(30490, 8)

In [75]:
final.to_csv("full.csv",index=False)

In [66]:
final['Discounted Price']=final['Original Price']*(1-0.1)
final['Predicted Quantity']=final['Original Quantity']*(1+final['Predicted PED']*(final['Discounted Price']-final['Original Price'])/final['Original Price'])

In [72]:
final.head()

,Product ID,Category,State ID,Original Price,Discounted Price,Predicted PED,Original Quantity,Predicted Quantity
0,FOODS_1_001_CA_1,FOODS,CA_1,2.24,2.24,-2.969951,29,29.0
1,FOODS_1_001_CA_2,FOODS,CA_2,2.24,2.24,-3.586702,31,31.0
2,FOODS_1_001_CA_3,FOODS,CA_3,2.24,2.24,-1.592398,23,23.0
3,FOODS_1_001_CA_4,FOODS,CA_4,2.24,2.24,-2.441156,7,7.0
4,FOODS_1_001_TX_1,FOODS,TX_1,2.24,2.24,-2.014099,1,1.0
